In [69]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch import nn
import pandas as pd
import numpy as np

In [70]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NN_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(13, 13),
            nn.LeakyReLU(),
            nn.Dropout(0,3),
            nn.Linear(13, 6),
            nn.LeakyReLU(),
            nn.Dropout(0,3),
            nn.Linear(6, 6),
            nn.LeakyReLU(),
            nn.Dropout(0,3),
            nn.Linear(6, 2)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


Using cpu device


In [71]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
def test(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [72]:
batch_size =64
epochs =50
lr= 0.003

def main(batch_size, epochs, lr):    
    heart=np.genfromtxt("Data/heart.csv", delimiter=",", dtype=np.float32, usemask=True, skip_header=1)

    X = heart.data[:,:-1]


    heart=np.genfromtxt("Data/heart.csv", delimiter=",", dtype=np.long, usemask=True, skip_header=1)
    y = heart[:,-1]


    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)

    dataset = TensorDataset(X, y)

    lengths = [int(len(dataset) * 0.8), len(dataset) - int(len(dataset) * 0.8)]
    train_dataset, test_dataset = random_split(dataset, lengths)

 
    
    test_loader = DataLoader(test_dataset, batch_size, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    
    model = NN_model().to(device)
    
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_loader, model, loss_fn, optimizer)
        test(test_loader, model, loss_fn)
    print("Done!")
    
    torch.save(model.state_dict(), 'model.pth')
    
main(batch_size, epochs, lr)

Epoch 1
-------------------------------
loss: 1.203417  [   64/  820]
Test Error: 
 Accuracy: 45.9%, Avg loss: 0.772735 

Epoch 2
-------------------------------
loss: 0.699889  [   64/  820]
Test Error: 
 Accuracy: 57.6%, Avg loss: 0.753051 

Epoch 3
-------------------------------
loss: 0.669232  [   64/  820]
Test Error: 
 Accuracy: 59.0%, Avg loss: 0.666196 

Epoch 4
-------------------------------
loss: 0.655088  [   64/  820]
Test Error: 
 Accuracy: 56.1%, Avg loss: 0.619474 

Epoch 5
-------------------------------
loss: 0.763214  [   64/  820]
Test Error: 
 Accuracy: 57.6%, Avg loss: 0.667822 

Epoch 6
-------------------------------
loss: 0.642025  [   64/  820]
Test Error: 
 Accuracy: 63.4%, Avg loss: 0.649221 

Epoch 7
-------------------------------
loss: 0.636972  [   64/  820]
Test Error: 
 Accuracy: 64.4%, Avg loss: 0.602688 

Epoch 8
-------------------------------
loss: 0.634338  [   64/  820]
Test Error: 
 Accuracy: 67.3%, Avg loss: 0.661235 

Epoch 9
----------------